<center> 
    <h1> GANs &#119070; &nbsp;Reels </h1>
</center>

## Demo ##

This notebook showcases our entire process of creating music using a Neural Network. At a high level, there are three main parts to our process of using a GAN to generate music. Namely:
1. Encoding: 
    Converting our dataset of Irish music to a form suitable for the GAN.
2. Training: 
    Feeding the curated data to our customized GAN, and decoding the results.
3. Decoding: 
    Converting the results from our GAN to music 

### 1. Encoding ###

This process can be broken down to 4 stages:
1. Sourcing: Getting the dataset from TheSessions.org/
2. Filtering: Pruning data that isn't in our relevant structure 
3. Cleaning: Using a structured notation to simplify ABC notation
4. Vectorizing: Creating vectors out of the cleaned data


In [1]:
# Entire Encoding Process
%matplotlib inline
from src.Generation import raw_to_npy as raw_data
raw_data.raw_abc_to_npy_file(update=False)

ModuleNotFoundError: No module named 'src'

### 2. Training ###

Our next step is to feed this curated data to our neural network. We opted for a Deconvolutional GAN since they are renowned for performing well on images, and making training a lot more stable. The parameters used were:
- Stride: 2
- Output function: Sigmoid
- Padding: 4 x 4 x 16
- Batch Size: 60

At a high level, this process consists of:
1. Building a Generator and a Discriminator
2. Padding and transposing data to suit our GAN
3. Checking for mode collapse every fixed number of iterations
4. Converting output of the GAN to a Vectorized array

Since training a Generative Adversarial Network takes a lot of computing power, we trained ours using Google Colab [found here](https://github.com/vin-nag/GANs-n-reels/blob/master/src/Model/MusicGAN.ipynb)

#### Using the Generated GAN ####

Once we trained our network, we use it to generate music

In [ ]:
import keras
trained_GAN = keras.models.load_model("../src/Model/Trained/generator.h5")

In [ ]:
import numpy as np
noise = np.random.normal(0, 1, [1, 100]) #20 arrays of noise of shape [100,]
generated_samples = trained_GAN.predict(noise)

In [ ]:
generated_samples = generated_samples[:,:-1, 4:-4]
generated_samples = np.squeeze(generated_samples, axis=3) #Remove colour channel
generated_samples = generated_samples.reshape([-1, 16, 16])

In [ ]:
MAX_PITCH = 80
MIN_PITCH = 53
MID_PITCH = (MAX_PITCH + MIN_PITCH) / 2
RANGE = MAX_PITCH - MIN_PITCH

#Map from [-1, 1] to [MIN_PITCH, MAX_PITCH]
generated_samples = (RANGE * generated_samples) + MID_PITCH

#Make values discrete
generated_samples = generated_samples.astype(np.int32)

### 3. Decoding ###

After the training is done, we decode the resulting vector (provided the GAN didn't collapse) back to music, using essentially the inverse of our encoding process.

This consists of:
1. Converting Vectors to ABC Notation
2. Converting ABC Notation to a format readable by a Music Player

In [ ]:
%matplotlib inline
# Decoding Process
# import matplotlib as plt
from src.Generation.Vectorizing import Decoding

decoder = Decoding.Decoder.from_single_vector(generated_samples, presentation=True)

decoder.play()

In [ ]:
# An interesting tune generated earlier
num = 7

generated = Decoding.Decoder.from_single_vector('generated_samples.npy', presentation=True)
decoder.play(num)